In [2]:
!pip install xgboost
import numpy as np
import pandas as pd
from xgboost import XGBRegressor
import pickle
import glob
import os

file_path = '/content/realty_data.csv'
titanic_data = pd.read_csv(file_path)

display(titanic_data.head(3))

,product_name,period,price,postcode,address_name,lat,lon,object_type,total_square,rooms,floor,city,settlement,district,area,description,source
0,"3-комнатная, 137 м²",NaN,63000000,127473.0,"2-й Щемиловский переулок, 5а",55.778894,37.608844,Квартира,137.0,3.0,6.0,Москва,NaN,Тверской район,NaN,Просторная квартира свободной планировки с пан...,ЦИАН
1,"Студия, 16,7 м²",NaN,3250000,108815.0,"Харлампиева, 46",55.551025,37.313054,Квартира,16.7,NaN,1.0,Москва,NaN,Филимонковское поселение,NaN,ВНИМАНИЕ! ОЧЕНЬ ПРИВЛЕКАТЕЛЬНОЕ ПРЕ...,Домклик
2,"3-комнатная, 76 м²",NaN,16004680,NaN,"ЖК Прокшино, 8 к4",55.594802,37.431264,Квартира,76.0,3.0,6.0,Москва,NaN,Сосенское поселение,NaN,"Apт.1684018. 0,01% - гибкая ипотека! Воспользу...",Яндекс.Недвижимость


In [4]:
print('Информация по таблице:')
display(titanic_data.info())

print('Cтатистические данные количественных признаков:')
print(titanic_data.describe(percentiles=None, include=[np.number]))

Информация по таблице:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98822 entries, 0 to 98821
Data columns (total 17 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   product_name  98822 non-null  object 
 1   period        0 non-null      float64
 2   price         98822 non-null  int64  
 3   postcode      93675 non-null  float64
 4   address_name  98821 non-null  object 
 5   lat           98822 non-null  float64
 6   lon           98822 non-null  float64
 7   object_type   98822 non-null  object 
 8   total_square  98822 non-null  float64
 9   rooms         94840 non-null  float64
 10  floor         98822 non-null  float64
 11  city          91928 non-null  object 
 12  settlement    6894 non-null   object 
 13  district      75111 non-null  object 
 14  area          19498 non-null  object 
 15  description   98573 non-null  object 
 16  source        98822 non-null  object 
dtypes: float64(7), int64(1), object(9)
memory usag

None

Cтатистические данные количественных признаков:
       period         price       postcode           lat           lon  \
count     0.0  9.882200e+04   93675.000000  98822.000000  98822.000000   
mean      NaN  2.512122e+07  124503.585119     55.742691     37.586404   
std       NaN  3.607234e+07   11956.742109      0.107044      0.169843   
min       NaN  1.900000e+06  101000.000000     55.468426     37.136489   
25%       NaN  1.050000e+07  115516.000000     55.673101     37.471611   
50%       NaN  1.516713e+07  123154.000000     55.745474     37.569365   
75%       NaN  2.500000e+07  140003.000000     55.817697     37.689568   
max       NaN  1.155219e+09  143989.000000     56.028824     38.122467   

       total_square         rooms         floor  
count  98822.000000  94840.000000  98822.000000  
mean      66.092176      2.197427      9.905274  
std       48.816204      1.038628      8.219180  
min        8.000000      1.000000      1.000000  
25%       40.100000      1.000000  

In [5]:
print(titanic_data['object_type'].unique())

['Квартира']


In [6]:
titanic_data = titanic_data.drop(columns=['object_type', 'product_name', 'period', 'address_name', 'city', 'settlement',
                          'district', 'area', 'description', 'source', 'postcode', 'lat', 'lon'])

In [7]:
titanic_data = titanic_data.dropna()
display(titanic_data.head(3))

,price,total_square,rooms,floor
0,63000000,137.0,3.0,6.0
2,16004680,76.0,3.0,6.0
3,7841776,24.0,1.0,10.0


In [8]:
X = titanic_data.drop(columns='price')
y = titanic_data['price']

In [9]:
model = XGBRegressor(n_estimators=300, learning_rate=0.05, random_state=13)
model.fit(X, y)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.05, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=300, n_jobs=None,
             num_parallel_tree=None, random_state=13, ...)

In [10]:
with open("real_estate_model.pkl", "wb") as f:
    pickle.dump(model, f)

In [11]:
%%writefile predict.py
import pickle


def load_model():
    with open("real_estate_model.pkl", "rb") as f:
        model = pickle.load(f)
    return model

def predict_price(features):
    model = load_model()
    price = model.predict([features])
    return float(round(price[0],0))

Writing predict.py


In [12]:
from predict import predict_price

print("Предсказанная цена:", predict_price([30, 1, 1]))
print("Предсказанная цена:", predict_price([30, 1, 5]))
print("Предсказанная цена:", predict_price([50, 1, 5]))
print("Предсказанная цена:", predict_price([50, 2, 5]))

Предсказанная цена: 7678184.0
Предсказанная цена: 8784718.0
Предсказанная цена: 18099912.0
Предсказанная цена: 14920369.0


In [13]:
!pip install fastapi uvicorn pyngrok
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from pyngrok import ngrok
from predict import predict_price
import uvicorn
import nest_asyncio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.7/94.7 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.3/73.3 kB 3.0 MB/s eta 0:00:00


In [ ]:
app = FastAPI()
nest_asyncio.apply()

class PredictionRequest(BaseModel):
    total_square: float
    rooms: int
    floor: int

@app.post("/post_test")
async def post_test_api(request: PredictionRequest):
    try:
        features = [request.total_square, request.rooms, request.floor]
        return {"features": features}
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

@app.post("/predict_post")
async def pred_price (request: PredictionRequest):
    try:
        features = [request.total_square, request.rooms, request.floor]
        print(features)
        prediction = predict_price(features)
        return {"predicted_price": prediction}
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

@app.get("/health")
async def health_check():
    return {"status": "alive"}

auth_token = "ваш_скопированный_токен"
ngrok.set_auth_token(auth_token)
ngrok_tunnel = ngrok.connect(8080)
print('Public URL:', ngrok_tunnel.public_url)

uvicorn.run(app, port=8080)